In [ ]:
import os
import s3fs

## Step 1: Get list of ICESat GLAH files

We use earthaccess, which uses NASA CMR, to query and get a list of files

In [9]:
import earthaccess

auth = earthaccess.login()


results = earthaccess.search_data(
    short_name='GLAH12',
    cloud_hosted=True
)
  

# if the data set is cloud hosted there will be S3 links available. The access parameter accepts "direct" or "external", direct access is only possible if you are in the us-west-2 region in the cloud.
data_links = [granule.data_links(access="direct") for granule in results]

# # or if the data is an on-prem dataset
# data_links = [granule.data_links(access="external") for granule in results]


We are already authenticated with NASA EDL
Granules found: 637


# Step 2: Get list of CmCt (Greenland) files

## Loop through CmCt dataset files

In [10]:
# Get NSIDC credentials
s3_cred_endpoint = {
    'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
    'gesdisc': 'https://data.gesdisc.earthdata.nasa.gov/s3credentials',
    'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials',
    'ornldaac': 'https://data.ornldaac.earthdata.nasa.gov/s3credentials',
    'ghrcdaac': 'https://data.ghrc.earthdata.nasa.gov/s3credentials',
    'nsidc': 'https://data.nsidc.earthdatacloud.nasa.gov/s3credentials'
}



In [11]:
import requests

def get_temp_creds(provider):
    try:
        response = requests.get(s3_cred_endpoint[provider])
        response.raise_for_status()  # This will raise an error if the HTTP request returned an error status code
        return response.json()  # Assuming the response is in JSON format
    except requests.RequestException as e:
        print(f"Failed to get credentials for {provider}. Error: {e}")
        return None

temp_creds_req = get_temp_creds('nsidc')


In [12]:
# Open connection to S3 using credentials

if temp_creds_req and 'accessKeyId' in temp_creds_req:
    fs_s3 = s3fs.S3FileSystem(anon=False, 
                              key=temp_creds_req['accessKeyId'], 
                              secret=temp_creds_req['secretAccessKey'], 
                              token=temp_creds_req['sessionToken'])
else:
    print("Failed to obtain valid S3 credentials.")

In [13]:
import glob

cmct_datasets_dir = '/efs/CmCt/datasets/'
year = '2003'
cmct_files = glob.glob(cmct_datasets_dir + '/GLAS_Data_orig/' + year + '/*.nc')#greenland
# print(cmct_files)


In [14]:
import raster


gimp_dem_filename = cmct_datasets_dir + "/GIMP_data/gimpdem_90m_v01.1.tif"
gimp_mask_filename = cmct_datasets_dir + "/GIMP_data/GimpIceMask_90m_2015_v1.2.tif"


gimp_dem = raster.readRasterBandAsArray(gimp_dem_filename, 1)
gimp_dem_gt = raster.getCoordinates(gimp_dem_filename, 1)

gimp_mask = raster.readRasterBandAsArray(gimp_mask_filename, 1)
gimp_mask_gt = raster.getCoordinates(gimp_mask_filename, 1)


/home/jovyan/users_conda_envs/cmct_invt/lib/python3.9/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [15]:
import rasterio


# Reading the raster file to extract its pixel size
with rasterio.open(gimp_dem_filename) as src:
    # Pixel width and height (cell size in x and y)
    pixel_size_x = src.transform[0]
    pixel_size_y = -src.transform[4]  # Negative because pixel sizes are usually stored as negative values

pixel_size_x, pixel_size_y


(90.0, 90.0)

In [16]:
import numpy as np

# Evaluate gradient in two dimensions
#cellsize_x and cellsize_y are both 90 meters
px, py = np.gradient(gimp_dem, 90,90)
slope = np.sqrt(px ** 2 + py ** 2)

#in degrees, convert using
slope_deg = np.degrees(np.arctan(slope))


In [17]:
from bilinear_interpolate import *

def sampleRasterAtPoint(rasterArray, geoTransform, xs, ys, method='bilinear', nodataValue=np.nan):
    zs = np.full(xs.shape, np.nan)  # Assuming xs and ys have the same shape

    for i, (x, y) in enumerate(zip(xs.flat, ys.flat)):
        imagex = (x - geoTransform[0]) / geoTransform[1]
        imagey = (y - geoTransform[3]) / geoTransform[5]

        if (0 <= imagex < rasterArray.shape[1]) and (0 <= imagey < rasterArray.shape[0]):
            if method == 'nearest':
                zs.flat[i] = rasterArray[int(np.floor(imagey)), int(np.floor(imagex))]
            elif method == 'bilinear':
                zs.flat[i] = bilinear_interpolate(rasterArray, imagex, imagey, nodataValue=nodataValue)

            if zs.flat[i] == nodataValue:
                zs.flat[i] = np.nan

    return zs

In [18]:
import numpy as np


def sample_gimp_dem_mask(gimp_dem, gimp_mask,slope_deg, gimp_dem_gt, gimp_mask_gt, xs, ys):
    
    # Check if xs and ys are single numbers (scalars)
    if np.isscalar(xs) and np.isscalar(ys):
        # Directly sample the raster at the single point
        gimp_dem_sampled = sampleRasterAtPoint(gimp_dem, gimp_dem_gt, xs, ys, method='bilinear')
        slope_deg_sampled = sampleRasterAtPoint(slope_deg, gimp_dem_gt, xs, ys, method='bilinear')
        gimp_mask_sampled = sampleRasterAtPoint(gimp_mask, gimp_mask_gt, xs, ys, method='nearest')
        return gimp_dem_sampled, gimp_mask_sampled
    
    # Otherwise, handle them as arrays
    gimp_dem_sampled = np.empty_like(xs, dtype=np.float64)
    slope_deg_sampled= np.empty_like(xs, dtype=np.float64)
    gimp_mask_sampled = np.empty_like(ys, dtype=np.float64)
    
    # Use np.ndindex for multidimensional index iteration
    for index in np.ndindex(xs.shape):
        x = xs[index]
        y = ys[index]
        gimp_dem_sampled[index] = sampleRasterAtPoint(gimp_dem, gimp_dem_gt, x, y, method='bilinear')
        slope_deg_sampled[index] = sampleRasterAtPoint(slope_deg, gimp_dem_gt, x, y, method='bilinear')
        gimp_mask_sampled[index] = sampleRasterAtPoint(gimp_mask, gimp_mask_gt, x, y, method='nearest')
    
    return gimp_dem_sampled, gimp_mask_sampled,slope_deg_sampled





In [19]:
# Function to find the drainage system for a given point
def find_drainage_system(lat,lon, polygons):
    point = Point(lat, lon)
    for drainage_id, polygon in polygons.items():
        if polygon.contains(point):
            return drainage_id
    return None  # Or some default value if the point doesn't fall in any polygon



#### Saving the filtered data to .nc file

In [20]:
import netCDF4 as nc
import numpy as np

def write_data_to_ncfile(nc_filename, time, lat_filtered, lon_filtered, elev_filtered, WGS84ELEV_M_filtered, EGM08MEANTIDEELEV_M_filtered, EGM08TIDEFREEELEV_M_filtered, ELEV_STDDEV_M_filtered, SLOPE_DEG_filtered, reflectivity_filtered, uncertainty_filtered, reference_elev_filtered, instrument_gain_filtered, gimp_dem_elev, drainage_ids, Icetype_filtered):
    with nc.Dataset(nc_filename, 'w', format='NETCDF4') as root_grp:
        
        
        root_grp.createDimension('t', None)  # Unlimited dimension
        root_grp.createDimension('phony_dim_1', len(lat_filtered))

        # Create variables
        dset_time = root_grp.createVariable('TIME', 'f8', ('t',), fill_value=9.96920996838687e+36)
        dset_lat = root_grp.createVariable('LAT_DEGN', 'f8', ('t'),fill_value=9.96920996838687e+36)
        dset_lon = root_grp.createVariable('LON_DEGE', 'f8', ('t',),fill_value=9.96920996838687e+36)
        dset_TOPEXELEV = root_grp.createVariable('TOPEXELEV_M', 'f4', ('t',),fill_value=9.96921e+36)

        
        ##check it again      
        dset_WGS84ELEV = root_grp.createVariable('WGS84ELEV_M', 'f4', ('t',),fill_value=9.96921e+36)
        dset_EGM08MEANTIDE = root_grp.createVariable('EGM08MEANTIDEELEV_M','f4', ('t',) ,fill_value=9.96921e+36)
        dset_EGM08TIDE = root_grp.createVariable('EGM08TIDEFREEELEV_M','f4', ('t',),fill_value= 9.96921e+36)
        dset_ELEV_STDDEV = root_grp.createVariable('ELEV_STDDEV_M','f4', ('t',),fill_value= 9.96921e+36 )
        dset_SLOPE_DEG = root_grp.createVariable('SLOPE_DEG','f4', ('t',),fill_value= 9.96921e+36 )
             

        
        dset_REFL_UNCORR = root_grp.createVariable('REFL_UNCORR', 'f4', ('t',),fill_value= 9.96921e+36)
        dset_WF_FIT_SIGMA_M = root_grp.createVariable('WF_FIT_SIGMA_M', 'f4', ('t',),fill_value= 9.96921e+36)        
        dset_GAIN = root_grp.createVariable('GAIN', 'i4', ('t',),fill_value=-2147483647)     
        dset_DEM_M= root_grp.createVariable('DEM_M', 'i4', ('t',),fill_value=-2147483647)
        dset_DRAINAGESYSTEM = root_grp.createVariable('DRAINAGESYSTEM','f4', ('t',),fill_value=9.96921e+36 )
        dset_ICETYPE = root_grp.createVariable('ICETYPE', 'i4', ('t',),fill_value=-2147483647 )        
        
              

        # Set data
        dset_time[:] = time
        dset_lat[:] = lat_filtered
        dset_lon[:] = lon_filtered
        dset_TOPEXELEV[:] = elev_filtered
        
        dset_WGS84ELEV[:]=WGS84ELEV_M_filtered
        dset_EGM08MEANTIDE[:]=EGM08MEANTIDEELEV_M_filtered
        dset_EGM08TIDE[:]=EGM08TIDEFREEELEV_M_filtered
        dset_ELEV_STDDEV[:]=ELEV_STDDEV_M_filtered
        dset_SLOPE_DEG[:]=SLOPE_DEG_filtered
        
        dset_REFL_UNCORR[:]=reflectivity_filtered
        dset_WF_FIT_SIGMA_M[:]=uncertainty_filtered
        dset_GAIN[:]=instrument_gain_filtered
        dset_DEM_M[:]=gimp_dem_elev
        dset_DRAINAGESYSTEM[:]=drainage_ids
        dset_ICETYPE[:]=Icetype_filtered
        


        # Set attributes for TIME
        dset_time.setncattr_string('units', "seconds")
        dset_time.setncattr_string('long_name', "UTC seconds since 2000-01-01 00:00:00")
        dset_time.setncattr_string('standard_name' ,"time")
     
        
        # Set attributes for LAT_DEGN
        dset_lat.setncattr_string('units',"degrees_north")
        dset_lat.setncattr_string('long_name', "latitude_north")
        dset_lat.setncattr_string('standard_name' , "latitude")
        dset_lat.valid_min = 59
        dset_lat.valid_max = 83.5
      

        # Set attributes for LON_DEGE
        dset_lon.setncattr_string('units',"degrees_east")
        dset_lon.setncattr_string('long_name', "longitude_east")
        dset_lon.setncattr_string('standard_name' , "longitude")
        dset_lon.valid_min = 285
        dset_lon.valid_max = 350
    
        
        
         # Set attributes 
        dset_TOPEXELEV.setncattr_string('units', "meters")
        dset_TOPEXELEV.setncattr_string('long_name', "Elevation relative to TOPEX/Poseidon ellipsoid")
        dset_TOPEXELEV.setncattr_string('standard_name' , "height_above_reference_ellipsoid" )
        dset_TOPEXELEV.valid_min = 0 
        dset_TOPEXELEV.valid_max= 3500 
  
        
        dset_WGS84ELEV.setncattr_string('units', "meters" )
        dset_WGS84ELEV.setncattr_string('long_name', "Elevation relative to WGS84 ellipsoid") 
        dset_WGS84ELEV.setncattr_string('standard_name', "height_above_reference_ellipsoid") 
        dset_WGS84ELEV.valid_min = 0 
        dset_WGS84ELEV.valid_max = 3500 
 
        
        dset_EGM08MEANTIDE.setncattr_string('units', "meters" )
        dset_EGM08MEANTIDE.setncattr_string('long_name', "Elevation relative to the EGM08 mean-tide geoid")
        dset_EGM08MEANTIDE.setncattr_string('standard_name' , "height_above_reference_geoid" )
        dset_EGM08MEANTIDE.valid_min = -150 
        dset_EGM08MEANTIDE.valid_max = 3500 
     
        
        dset_EGM08TIDE.setncattr_string('units', "meters" )
        dset_EGM08TIDE.setncattr_string('long_name',"Elevation relative to the EGM08 tide-free geoid")
        dset_EGM08TIDE.setncattr_string('standard_name', "height_above_reference_geoid")
        dset_EGM08TIDE.valid_min = -150 
        dset_EGM08TIDE.valid_max = 3500 
   
        
        dset_ELEV_STDDEV.setncattr_string('units', "meters")
        dset_ELEV_STDDEV.setncattr_string('long_name', "Uncertainty in elevation based on unpublished analysis by John Dimarzio (john.dimarzio@nasa.gov). Uncertainties are a function of GLAS campaign period and surface slope" )
        dset_ELEV_STDDEV.setncattr_string('nonstandard_name',"1_standard_deviation_height_uncertainty" )
        dset_ELEV_STDDEV.setncattr_string('coverage_content_type',  "qualityInformation" )
        dset_ELEV_STDDEV.valid_min = 0.
        dset_ELEV_STDDEV.valid_max = 0.5
       
        
        dset_SLOPE_DEG.setncattr_string('units', "meters" )
        dset_SLOPE_DEG.setncattr_string('long_name', "surface_slope at nearest node from GLAS elevation grid")
        dset_SLOPE_DEG.setncattr_string('nonstandard_name' , "slope" )
        dset_SLOPE_DEG.valid_min = 0.
        dset_SLOPE_DEG.valid_max = 0.5 
  
        dset_REFL_UNCORR.setncattr_string('units', "dimensionless" )
        dset_REFL_UNCORR.setncattr_string('long_name', "Uncorrected reflectivity from the GLAS data records" )
        dset_REFL_UNCORR.setncattr_string('nonstandard_name' ,"surface_reflectivity" )
        dset_REFL_UNCORR.valid_min = 0.
        dset_REFL_UNCORR.valid_max = 2. 

        dset_WF_FIT_SIGMA_M.setncattr_string('units',"volts")
        dset_WF_FIT_SIGMA_M.setncattr_string('long_name',"Standard deviation of the fit to the waveform from GLAS data record") 
        dset_WF_FIT_SIGMA_M.setncattr_string('nonstandard_name' ,  "uncertainty") 
        dset_WF_FIT_SIGMA_M.setncattr_string('coverage_content_type', "qualityInformation")       
        dset_WF_FIT_SIGMA_M.valid_min = 0. 
        dset_WF_FIT_SIGMA_M.valid_max = 2.
   

        dset_GAIN.setncattr_string('units', "dimensionless ")
        dset_GAIN.setncattr_string('long_name', "Instrument gain from the GLAS data record")
        dset_GAIN.setncattr_string('nonstandard_name' ,  "instrument_gain")
        dset_GAIN.setncattr_string('coverage_content_type', "qualityInformation")
        dset_GAIN.valid_min = 1 
        dset_GAIN.valid_max = 250
   

        dset_DEM_M.setncattr_string('units', "meters") 
        dset_DEM_M.setncattr_string('long_name',"height relative to WGS84 at nearest node of the reference DEM")
        dset_DEM_M.setncattr_string('standard_name', "height_above_reference_ellipsoid") 
        dset_DEM_M.setncattr_string('coverage_content_type',  "qualityInformation")
        dset_DEM_M.valid_min = 0
        dset_DEM_M.valid_max = 3500


        
        dset_DRAINAGESYSTEM.setncattr_string('units', "dimensionless ")
        dset_DRAINAGESYSTEM.setncattr_string('long_name',"Drainage system or subsystem based on Mario Giovinetto\'s analysis of ICESat/GLAS data")
        dset_DRAINAGESYSTEM.setncattr_string('nonstandard_name' , "drainage_system_id")
        dset_DRAINAGESYSTEM.setncattr_string('reference_url', "http://icesat4.gsfc.nasa.gov/cryo_data/ant_grn_drainage_systems.php") 
        dset_DRAINAGESYSTEM.valid_min= 0
        dset_DRAINAGESYSTEM.valid_max = 27

        
        
        dset_ICETYPE.setncattr_string('units', "dimensionless ")
        dset_ICETYPE.setncattr_string('long_name', "Ice type ")
        dset_ICETYPE.setncattr_string('comment_1', "0: isolated ice cap (Greenland only). ")
        dset_ICETYPE.setncattr_string('comment_2', "1: coterminous ice sheet. ")
        dset_ICETYPE.setncattr_string('comment_3', "2: ice shelf (Antarctica only). ")
        dset_ICETYPE.setncattr_string('nonstandard_name', "icetype_flag ")
        dset_ICETYPE.setncattr_string('reference_url', "http://nsidc.org/data/docs/agdc/nsidc0489_bindschadler/ ")
        dset_ICETYPE.setncattr_string('reference', "The Cryosphere, 5, 569–588, 2011. doi:10.5194/tc-5-569-2011")                          
        dset_ICETYPE.valid_min = 0
        dset_ICETYPE.valid_max = 2        
        

        

        # Set global attributes
        root_grp.setncattr_string('id',"GLA12_634_2131_002_0057_0_01_0001.CMCT.final.nc")
        root_grp.setncattr_string('date_created', "2016-Jun-22 UTC")
        root_grp.setncattr_string('product_version',"R634")
        root_grp.setncattr_string('history', "GLA12_634_2131_002_0057_0_01_0001.CMCT.final.nc created Wed Jun 22 19:44:41 2016 by Saba using make_glas_cmct_netcdf.pro, IDL 8.4 run on gs6141-atlas. Input file: /data1/jack/CMCT/GLAS_Data/R634_Greenland/L2F/GLA12_634_2131_002_0057_0_01_0001.CMCT.final.dat")
        root_grp.setncattr_string('keywords', "altimetry, glaciology, modeling, Greenland")
        root_grp.setncattr_string('geographic_identifier', "Greenland")
        
        
        root_grp.setncattr('geospatial_lat_min', np.int32(59))
        root_grp.setncattr('geospatial_lat_max', np.float32(83.5))
        root_grp.setncattr('geospatial_lon_min', np.int32(285))
        root_grp.setncattr('geospatial_lon_max', np.int32(350))
        root_grp.setncattr('geospatial_vertical_max', np.int32(3500))

        
        root_grp.setncattr_string('creator_name', "Jack Saba")
        root_grp.setncattr_string('creator_email', "jack.saba@nasa.gov")
        root_grp.setncattr_string('institution', "NASA/GSFC Code 615")
        root_grp.setncattr_string('project', "Cryospheric Model Comparison Tool")
        root_grp.setncattr_string('publisher_name_1', "Thomas Neumann" )
        root_grp.setncattr_string('publisher_name_2', "Sophie Nowicki" )                         
        root_grp.setncattr_string('publisher_email_1', "Thomas.Neumann@nasa.gov")
        root_grp.setncattr_string('publisher_email_2',"sophie.nowicki@nasa.gov" )
        root_grp.setncattr_string('geospatial_lat_units', "degrees_north")                        
        root_grp.setncattr_string('geospatial_lon_units', "degrees_east")
        root_grp.setncattr_string('geospatial_vertical_units_1', "height above the TOPEX/Poseidon ellipsoid, in meters")
        root_grp.setncattr_string('geospatial_vertical_units_2', "EPSG:4979 (height above the WGS84 ellipsoid, in meters)" )
        root_grp.setncattr_string('geospatial_vertical_units_3', "height above the EGM08 mean-tide geoid, in meters")
        root_grp.setncattr_string('geospatial_vertical_units_4', "height above the EGM08 tide-free geoid, in meters")
        root_grp.setncattr_string('geospatial_vertical_min_1', "0" )                         
        root_grp.setncattr_string('geospatial_vertical_min_2', "0" )  
        root_grp.setncattr_string('geospatial_vertical_min_3', "0" )                         
        root_grp.setncattr_string('geospatial_vertical_min_4', "0")
        root_grp.setncattr_string('geospatial_bounds_crs', "EPSG:4979 (WGS84)" )                        
        root_grp.setncattr_string('naming_authority', "nasa/gsfc/code_615" ) 
                           
        root_grp.setncattr_string('references_1',"Bamber, Jonathan L., Jose Luis Gomez-Dans, and Jennifer A. Griggs. 2009. Antarctic 1 km Digital Elevation Model (DEM) from Combined ERS-1 Radar and ICESat Laser Satellite Altimetry. Boulder, Colorado USA: National Snow and Ice Data Center. Digital media. https://nsidc.org/data/docs/daac/nsidc0422_antarctic_1km_dem/")
        root_grp.setncattr_string('references_2' , "Bamber, J. L., J. L. Gomez-Dans, and J. A. Griggs. 2009. A New 1 km Digital Elevation Model of the Antarctic Derived from Combined Satellite Radar and Laser Data Part 1: Data and Methods. The Cryosphere, 3, 101-111")
        root_grp.setncattr_string('references_3',"Bindschadler, R., H. Choi, A. Wichlac2, R. Bingham, J. Bohlander, K. Brunt, H. Corr, R. Drews, H. Fricker, M. Hall, R. Hindmarsh, J. Kohler, L. Padman, W. Rack, G. Rotschky, S. Urbini, P. Vornberger, and N. Young, 2011, Getting around Antarctica: new high-resolution mappings of the grounded and freely-floating boundaries of the Antarctic ice sheet created for the International Polar Year, The Cryosphere, 5, 569–588, doi:10.5194/tc-5-569-2011a" )
        root_grp.setncattr_string('references_4' ,  "Bindschadler, R., H. Choi, and ASAID Collaborators. 2011b. High-resolution Image-derived Grounding and Hydrostatic Lines for the Antarctic Ice Sheet. Boulder, Colorado, USA: National Snow and Ice Data Center. http://dx.doi.org/10.7265/N56T0JK2" )
        root_grp.setncattr_string('references_5' ,  "Griggs, J. A. and J. L. Bamber. 2009. A New 1 km Digital Elevation Model of Antarctica Derived from Combined Radar and Laser Data Part 2: Validation and Error Estimates. The Cryosphere, 3, 113123" )
        root_grp.setncattr_string('references_6' , "Howat, I. M., A. Negrete, and B. E. Smith, 2014, The Greenland Ice Mapping Project (GIMP) land classification and surface elevation data sets, The Cryosphere, 8, 1509–1518, doi:10.5194/tc-8-1509-2014. See https/::bpcrc.osu.edu:gdg:data" )
        root_grp.setncattr_string('references_7' ,  "Howat, I., A. Negrete, and B. Smith. 2015. MEaSURES Greenland Ice Mapping Project (GIMP) Digital Elevation Model, Version 1. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. http://dx.doi.org/10.5067/NV34YUIXLP9W" )
        root_grp.setncattr_string('references_8' ,  "Pavlis, Nikolaos K, Simon A. Holmes, Steve C. Kenyon, and John K. Factor , The development and evaluation of the Earth Gravitational Model 2008 (EGM2008), 2012, JGR 117, B04406, doi:10.1029/2011JB008916. http://earth-info.nga.mil/GandG/wgs84/gravitymod/egm2008/" )
        root_grp.setncattr_string('references_9' , "Shepherd, Andrew, Erik R. Ivins, Geruo A, Valentina R. Barletta, Mike J. Bentley, Srinivas Bettadpur, Kate H. Briggs, David H. Bromwich, René Forsberg, Natalia Galin, Martin Horwath, Stan Jacobs, Ian Joughin, Matt A. King, Jan T. M. Lenaerts, Jilu Li, Stefan R. M. Ligtenberg, Adrian Luckman, Scott B. Luthcke, Malcolm McMillan, Rakia Meister, Glenn Milne, Jeremie Mouginot, Alan Muir, Julien P. Nicolas, John Paden, Antony J. Payne, Hamish Pritchard, Eric Rignot, Helmut Rott, Louise Sandberg Sørensen, Ted A. Scambos, Bernd Scheuchl, Ernst J. O. Schrama, Ben Smith, Aud V. Sundal, Jan H. van Angelen, Willem J. van de Berg, Michiel R. van den Broeke, David G. Vaughan, Isabella Velicogna, John Wahr, Pippa L. Whitehouse, Duncan J. Wingham, Donghui Yi, Duncan Young, and H. Jay Zwally, 2012, A Reconciled Estimate of Ice-Sheet Mass Balance, Science 338, 1183 (2012). doi:10.1126/science.1228102")

        root_grp.setncattr_string('processing_level' , "final") 
        root_grp.setncattr_string('comments', "The drainage systems (DS) were determined from grids based on the DS boundaries generated by Mario Giovinetto (http://icesat4.gsfc.nasa.gov/cryo_data/ant_grn_drainage_systems.php) were used to generate 500m (for Antarctica) and 1 km (for Greenland) DS ids the DS was assigned based on the DS of the nearest node. The DS grids were generated by John Robbins." )
        root_grp.setncattr_string('acknowledgement', "We acknowledge support from the NASA Cryospheric Sciences Program") 
        root_grp.setncattr_string('license', "Freely Distributed" )
        root_grp.setncattr_string('program',"Cryospheric Model Comparison Tool (CMCT)" )


        print(f"Data saved to {nc_filename}")




In [21]:
import pandas as pd
from shapely.geometry import Point, Polygon
from collections import defaultdict

# Assuming 'data' is your data from GrnDrainageSystems_Ekholm_epsg3413.txt
data = pd.read_csv('/efs/data/Ice_Sheet_Drainage_Systems/GrnDrainageSystems_Ekholm_epsg3413.txt', sep='\t', header=None, names=['x', 'y', 'drainage_id'])

# Group points by drainage system ID
grouped = data.groupby('drainage_id')

# Create polygons for each drainage system
polygons = {drainage_id: Polygon(points.values) for drainage_id, points in grouped[['x', 'y']]}



In [23]:
# import netCDF4 as nc
outputdir='/efs/CmCt/datasets/GLAS_Data/'+year+'/'

In [25]:
import h5py
import progressbar
from pyproj import Transformer
import numpy as np
import pandas as pd



all_data = []  # List to store individual DataFrames for each cmct file
bar = progressbar.ProgressBar()
for cmct_file in bar(cmct_files):

    #Greenland
    cmct_file_basename = cmct_file.split('/')[-1].split('.')[0].replace('GLA12','GLAH12')
        
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3413")#Greenland
    
    for data_link in data_links:
        try:
            if cmct_file_basename in data_link[0]:
                print(data_link[0])

                #read glah data
                f_glah = h5py.File(fs_s3.open(data_link[0], 'rb'))

                # Extract relevant datasets

                #time
                time_glah = f_glah['Data_40HZ']['Time']['d_UTCTime_40'][:]

                #a. Latitude 
                LAT_glah = f_glah['Data_40HZ']['Geolocation']['d_lat'][:]

                # b.Longitude
                LON_glah = f_glah['Data_40HZ']['Geolocation']['d_lon'][:]

                #c.Elevation 
                elev_glah = f_glah['Data_40HZ']['Elevation_Surfaces']['d_elev'][:]
             
                WGS84ELEV_M_glah = f_glah['Data_40HZ']['Elevation_Surfaces']['d_elev'][:] - f_glah['Data_40HZ']['Geophysical']['d_deltaEllip'][:]

                ## assining value(NaN/0.5) for these unknown variables
                size_of_array = elev_glah.shape
                # Creating arrays with the same size              
                EGM08MEANTIDEELEV_M_glah = np.full(size_of_array, np.nan)  # Filled with NaN
                EGM08TIDEFREEELEV_M_glah = np.full(size_of_array, np.nan)  # Filled with NaN            
                ELEV_STDDEV_M_glah = np.full(size_of_array, 0.5)           # Filled with 0.5


                #d. Reflectivity 
                reflectivity_glah = f_glah['Data_40HZ']['Reflectivity']['d_reflctUC'][:]

                #e. GLAS fitting uncertainty 
                uncertainty_glah = f_glah['Data_40HZ']['Elevation_Surfaces']['d_IceSVar'][:]


                #f. The reference elevation that was stored in the GLAS data records       
                reference_elev_glah = f_glah['Data_40HZ']['Geophysical']['d_DEM_elv'][:]

                #h.The instrument gain 
                instrument_gain_glah = f_glah['Data_40HZ']['Waveform']['i_gval_rcv'][:]



                #1. Project GLAS latitude/longitude into EPSG:3413 polar stereographic projection
                x, y = transformer.transform(LAT_glah, LON_glah)


                # Sample the GIMP ice mask and GIMP DEM elevation and slope for the projected coordinates
                gimp_dem_sampled, gimp_mask_sampled,slope_deg_sampled=sample_gimp_dem_mask(gimp_dem,gimp_mask,slope_deg,gimp_dem_gt,gimp_mask_gt, x, y)


                # 2. Filter out points not on ice
                on_ice_mask = (gimp_mask_sampled >= 1) & (gimp_mask_sampled <= 3)#if values are 1,2and 3

                #A flag indicating whether the data are on the ice sheet or an ice shelf
                # Icetype: 1 for ice shelf (gimp_mask_sampled = 3), 0 for ice sheet (gimp_mask_sampled = 1 or 2or 0)
                Icetype = np.where(gimp_mask_sampled == 3, 1, 0)                

                # 3. Filter out points with elevation difference <= 200          
                elev_diff_mask = np.abs(elev_glah - gimp_dem_sampled) <= 200

                # 4. Filter points based on GLAS surface reflectivity           
                reflectivity_mask = reflectivity_glah >= 0.0375

                # 5. Filter points based on GLAS elevation uncertaint
                uncertainty_mask = uncertainty_glah <= 0.0375

                

                # Combine all filters
                combined_mask = on_ice_mask & elev_diff_mask & reflectivity_mask & uncertainty_mask


                #6. Filter the data based on the combined mask
                time_filtered=time_glah[combined_mask]
                lat_filtered = LAT_glah[combined_mask]
                lon_filtered = LON_glah[combined_mask]
                elev_filtered = elev_glah[combined_mask]
                WGS84ELEV_M_filtered = WGS84ELEV_M_glah[combined_mask]

                EGM08MEANTIDEELEV_M_filtered = EGM08MEANTIDEELEV_M_glah[combined_mask] 
                EGM08TIDEFREEELEV_M_filtered = EGM08TIDEFREEELEV_M_glah[combined_mask]
                ELEV_STDDEV_M_filtered = ELEV_STDDEV_M_glah[combined_mask]
                
                SLOPE_DEG_filtered= slope_deg_sampled[combined_mask]
                reflectivity_filtered=reflectivity_glah[combined_mask]
                uncertainty_filtered=uncertainty_glah[combined_mask]
                reference_elev_filtered=reference_elev_glah[combined_mask]
                instrument_gain_filtered =instrument_gain_glah[combined_mask]


                #g.The elevation from the GIMP or Bamber DEM.            
                gimp_dem_elev=gimp_dem_sampled[combined_mask]

                #i.The drainage system ID            
                x_filtered=x[combined_mask]
                y_filtered=y[combined_mask]
                drainage_ids = [find_drainage_system(lat1, lon1, polygons) for lat1, lon1 in zip(x_filtered, y_filtered)]


                #j.A flag indicating whether the data are on the ice sheet or an ice shelf
                # Icetype: 1 for ice shelf (gimp_mask_sampled = 3), 0 for ice sheet (gimp_mask_sampled = 1 or 2)
                Icetype_filtered = Icetype[combined_mask]

                # Replace None with a specific numerical value, e.g., -1
                none_value = -1
                drainage_ids_with_value = [none_value if x is None else x for x in drainage_ids]
                # Name of output file
                # Split the string at each '/'
                parts = data_link[0].split('/')

                # The file name is the last element in the list
                file_name = parts[-1]

                # CmCt file name
                nc_fname = file_name.rsplit('.', 1)[0] + '.CMCT.final.nc'
                # print(nc_fname)

                # Create a new NetCDF file
                fname = f'{outputdir}'+ nc_fname## filename
                print(fname)


                # save the output data to .nc file
                write_data_to_ncfile(fname,time_filtered,lat_filtered,lon_filtered,elev_filtered,WGS84ELEV_M_filtered,EGM08MEANTIDEELEV_M_filtered,EGM08TIDEFREEELEV_M_filtered,ELEV_STDDEV_M_filtered,SLOPE_DEG_filtered,reflectivity_filtered,uncertainty_filtered,reference_elev_filtered,instrument_gain_filtered,gimp_dem_elev,drainage_ids_with_value,Icetype_filtered)
        except KeyError as e:
            print(f"KeyError encountered in file {data_link[0]}: {e}")
            continue  # Skip the rest of the loop and move to the next file






s3://nsidc-cumulus-prod-protected/GLAS/GLAH12/034/2003/11/GLAH12_634_2103_002_0337_0_01_0001.H5


/efs/CmCt/datasets/GLAS_Data/2003/GLAH12_634_2103_002_0337_0_01_0001.CMCT.final.nc
